In [1]:
from z3 import *
from itertools import combinations
from typing import Sequence

Read instance file:

In [2]:
input_filename = '../../Instances/15x15-ide.txt'

w, h, n, DX, DY = None, None, None, None, None

with open(input_filename, 'r') as f_in:
    lines = f_in.read().splitlines()

    split = lines[0].split(' ')
    w = int(split[0])
    h = int(split[1])

    n = int(lines[1])

    DX = []
    DY = []

    for i in range(int(n)):
        split = lines[i + 2].split(' ')
        DX.append(int(split[0]))
        DY.append(int(split[1]))

Solver:

In [3]:
solver = Solver()

Model:

In [4]:
XY = [(Int(f'XY_{i}_0'), Int(f'XY_{i}_1')) for i in range(n)]
XY

[(XY_0_0, XY_0_1),
 (XY_1_0, XY_1_1),
 (XY_2_0, XY_2_1),
 (XY_3_0, XY_3_1),
 (XY_4_0, XY_4_1),
 (XY_5_0, XY_5_1),
 (XY_6_0, XY_6_1),
 (XY_7_0, XY_7_1),
 (XY_8_0, XY_8_1),
 (XY_9_0, XY_9_1),
 (XY_10_0, XY_10_1),
 (XY_11_0, XY_11_1)]

Constraints:

In [5]:
# Non-overlapping constraint
for (i, j) in combinations(range(n), 2):
    solver.add(Or(XY[i][0] + DX[i] <= XY[j][0], 
                  XY[j][0] + DX[j] <= XY[i][0],
                  XY[i][1] + DY[i] <= XY[j][1],
                  XY[j][1] + DY[j] <= XY[i][1]))

In [6]:
# Boundaries consistency constraint
for i in range(n):
    solver.add(XY[i][0] >=0)
    solver.add(XY[i][1] >= 0)
    solver.add(XY[i][0] + DX[i] <= w)
    solver.add(XY[i][1] + DY[i] <= h)

In [7]:
# Cumulative constraint
def cumulative(solver, S: Sequence, D: Sequence, R: Sequence, C: int):
    # Iterate over the durations
    for u in D:
        solver.add(
            Sum(
                [If(And(S[i] <= u, u < S[i] + D[i]), R[i], 0) for i in range(n)]
            ) <= C)

# Implied constraints
cumulative(solver,
           S=list(map(lambda t: t[0], XY)),  # take x coordinates
           D=DX,
           R=DY,
           C=h)
cumulative(solver,
           S=list(map(lambda t: t[1], XY)),  # take y coordinates
           D=DY,
           R=DX,
           C=w)

Ordering constraint

In [8]:
def is_valid(x_i, y_i, x_j, y_j, dx, dy):
    right = And(x_j >= x_i + dx, y_j >= y_i)
    up = And(y_j >= y_i + dy, x_j >= x_i)
    return Or(right, up)

for (i, j) in combinations(range(n), 2):
    solver.add(Or(Not(And(DX[i] == DX[j], DY[i] == DY[j])), is_valid(XY[i][0], XY[i][1], XY[j][0], XY[j][1], DX[i], DY[i])))

In [9]:
%%time
solver.check()

CPU times: user 126 ms, sys: 4.59 ms, total: 131 ms
Wall time: 129 ms


sat

From Z3 model solution to file:

In [10]:
model = solver.model()

xy = [(model[XY[i][0]], model[XY[i][1]]) for i in range(n)]
xy

[(6, 9),
 (0, 4),
 (0, 0),
 (12, 10),
 (0, 6),
 (0, 8),
 (12, 0),
 (12, 8),
 (6, 0),
 (3, 0),
 (9, 0),
 (3, 12)]

In [11]:
output_filename = '../../pwp_utilities/15x15-ide.txt'
with open(output_filename, 'w') as f_out:
    f_out.write('{} {}\n'.format(w, h))
    f_out.write('{}\n'.format(n))
    for i in range(n):
        f_out.write('{} {}\t{} {}\n'.format(DX[i], DY[i], xy[i][0], xy[i][1]))